# xAPI-SG Data Generator

### Generator of traces xAPI data following the Experience API for Serious Games Profile (xAPI-SG)

**xAPI-SG main reference:**
*Applying standards to systematize learning analytics in serious games.
Ángel Serrano-Laguna, Iván Martínez-Ortiz, Jason Haag, Damon Regan, Andy Johnson, Baltasar Fernández-Manjón
Computer Standards & Interfaces 50 (2017) 116–123, http://dx.doi.org/10.1016/j.csi.2016.09.014*

Further info on GitHub wiki page: https://github.com/e-ucm/rage-analytics/wiki/xAPI-SG-Profile

The following code:
- expect you select a verb, a type_object, number of players, numbers of IDs, numbers of traces to generate
- generate a file with a list of xAPI-SG statements

In [1]:
import json
from datetime import datetime, timedelta
import copy
import math
from ipywidgets import interact, interactive, fixed, HBox, Layout,VBox
import ipywidgets as widgets
from IPython.display import display, clear_output
import pprint
import random
import uuid
import os

**Save generator data**


In [2]:
output=widgets.Output()

def save_generated_data(statement_dict, verbs, types):
    start_time = datetime.now()
    strtime=start_time.strftime("%Y-%m-%d-%H-%M-%S")
    filename="generated_"
    for v in verbs:
        filename+=v+"_"
    for t in types:
        filename+=t+"_"
    filename+=str(nbtraces.value)+"statements_"+strtime+".json"
    path=os.path.join(os.getcwd(),filename)
    with open(path, 'w', encoding='UTF-8') as file:
        file.write(json.dumps(statement_dict, indent=2))
    end_time = datetime.now()

**Template statements**

In [3]:
template_statement={
    "actor":{
        "id":None
    },
    "verb":{
        "id":"https://w3id.org/xapi/seriousgames/verbs/"
    },
    "object":{
        "id":None,
        "definition":{
            "type":"https://w3id.org/xapi/seriousgames/activity-types/"
        }
    },
    "timestamp":None
}

def generator(randomPlayer, randomVerb, randomType,randomID, time):
    statement = copy.deepcopy(template_statement)
    statement["actor"]["name"]=str(randomPlayer)
    statement["actor"]["id"]=str(randomPlayer)
    statement["verb"]["id"]+=randomVerb
    statement["object"]["definition"]["type"]+=randomType
    statement["object"]["id"]=str(randomID)
    statement["timestamp"]=time.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
    return statement

**Select number of traces, players and IDs** to generate

In [4]:
nbtraces = widgets.IntSlider(description="Nb traces", min=1, max=10000, continuous_update=False)
nbtracestext = widgets.IntText(description="", continuous_update=False)
widgets.link((nbtraces, 'value'), (nbtracestext, 'value'))
traces=VBox([nbtraces,nbtracestext])

nbplayers = widgets.IntSlider(description="Nb players", min=1, continuous_update=False)
nbplayerstext = widgets.IntText(description="", continuous_update=False)
widgets.link((nbplayers, 'value'), (nbplayerstext, 'value'))
widgets.link((nbtraces, 'value'), (nbplayers, 'max'))
players=VBox([nbplayers,nbplayerstext])

nbIDs = widgets.IntSlider(description="Nb IDs", min=1, continuous_update=False)
nbIDstext = widgets.IntText(description="", continuous_update=False)
widgets.link((nbIDs, 'value'), (nbIDstext, 'value'))
widgets.link((nbtraces, 'value'),(nbIDs, 'max'))
IDs=VBox([nbIDs,nbIDstext])
vbox_nb_traces_ID_player=VBox([traces,players, IDs])

**Generator** of players, IDs and times datas

In [5]:
def generate_players():
    tabplayers=[]
    for n in range(0, nbplayers.value):
        tabplayers.append(uuid.uuid4())
    return tabplayers

#print(generate_players())

def generate_IDs():
    tabIDs=[]
    for n in range(0, nbIDs.value):
        tabIDs.append(uuid.uuid4())
    return tabIDs

#print(generate_IDs())

def generate_times_datas():
    times=[]
    time = datetime.now()
    for n in range(0, nbtraces.value):
        times.append(time)
        timeToSleep = random.random()*100
        dtime=timedelta(seconds=timeToSleep)
        time+=dtime
    return times

#print(generate_times_datas())

**Multi checkboxes selector** for verbs and types

In [6]:
%run multipleSelectorCheckboxesWidget.ipynb
class GeneratorMultiCheckboxWidget(MultiCheckboxWidget):
    def update(self):
        pass

**Type Checker**

In [7]:
def checkType(checkType, randomID, randomPlayer, types):
    randomType=None
    for type in checkType.keys():
            if randomID in checkType[type].keys():
                randomType=type
    if randomType==None:
            randomType=random.choice(types)
            if randomType not in checkType.keys():
                checkType[randomType]={}
            checkType[randomType][randomID]={}
    if randomPlayer not in checkType[randomType][randomID].keys():
            checkType[randomType][randomID][randomPlayer]=None
    return randomType

**Completables**

In [8]:
types_completables=["serious-game", "game", "session", "level", "quest", "stage", "combat", "storynode", "race", "completable"]
checkboxes_types_completables=GeneratorMultiCheckboxWidget("Completables types", types_completables)

verbs_completables=["initialized", "progressed", "completed"]
checkboxes_verbs_completables=GeneratorMultiCheckboxWidget("Completables verbs", verbs_completables)

template_result_progress={
    "extensions": {
		"https://w3id.org/xapi/seriousgames/extensions/progress": None,
		"progress": None
	}
}

template_result_completed_score={
	"extensions": {
		"success": None,
		"score":None
	}
}

def generate_completable(change):
    start_time=datetime.now()
    with output:
        print("Starting the data generation")
    verbs=checkboxes_verbs_completables.get_selected_options()
    types=checkboxes_types_completables.get_selected_options()
    ids=generate_IDs()
    times=generate_times_datas()
    player=generate_players()
    statements=[]
    checkCompletable={}
    for i in range(nbtraces.value):
        randomID=random.choice(ids)
        randomPlayer=random.choice(player)
        randomType = checkType(checkCompletable,randomID,randomPlayer,types)
        if checkCompletable[randomType][randomID][randomPlayer] in [None, False]:
            checkCompletable[randomType][randomID][randomPlayer]=True
            randomVerb="initialized"
        else:
            randomVerb=random.choice(["progressed", "completed"])
            if randomVerb=="completed":
                checkCompletable[randomType][randomID][randomPlayer]=False
        time=times[i]
        statement=generator(randomPlayer, randomVerb, randomType,randomID, time)
        if randomVerb=="completed":
            result=copy.deepcopy(template_result_completed_score)
            result["extensions"]["score"]=random.random()*100
            result["extensions"]["success"]=random.choice([True,False])
            statement["result"]=result
        elif randomVerb=="progressed":
            result=copy.deepcopy(template_result_progress)
            progress=random.random()
            result["extensions"]["https://w3id.org/xapi/seriousgames/extensions/progress"]=progress
            result["extensions"]["progress"]=progress
            statement["result"]=result
        statements.append(statement)
    save_generated_data(statements, verbs, types)
    end_time=datetime.now()
    delta=end_time-start_time
    with output:
        print("End the data generation")
        print("Generated in "+str(delta))
        clear_output(wait=True)

buttonCompletable=widgets.Button(description='Generate Data')
buttonCompletable.on_click(generate_completable)

tabCompletable=VBox([
    HBox([
        checkboxes_verbs_completables.multi_select,
        checkboxes_types_completables.multi_select,
        vbox_nb_traces_ID_player
    ]),
    buttonCompletable
])

**Accessibles**

In [9]:
verbs_accessibles=["accessed", "skipped"]
checkboxes_verbs_accessibles=GeneratorMultiCheckboxWidget("Accessibles verbs", verbs_accessibles)

types_accessibles=["screen", "area", "zone", "cutscene", "accessible"]
checkboxes_types_accessibles=GeneratorMultiCheckboxWidget("Accessibles types", types_accessibles)

def generate_accessibles(change):
    start_time=datetime.now()
    with output:
        print("Starting the data generation")
    verbs=checkboxes_verbs_accessibles.get_selected_options()
    types=checkboxes_types_accessibles.get_selected_options()
    ids=generate_IDs()
    times=generate_times_datas()
    player=generate_players()
    statements=[]
    checkTypes={}
    for i in range(nbtraces.value):
        randomID=random.choice(ids)
        randomPlayer=random.choice(player)
        randomType = checkType(checkTypes,randomID,randomPlayer,types)
        if checkTypes[randomType][randomID][randomPlayer]==None:
            randomVerb="accessed"
            checkTypes[randomType][randomID][randomPlayer]=True
        else:
            randomVerb="skipped"
            checkTypes[randomType][randomID][randomPlayer]=None
        time=times[i]
        statement=generator(randomPlayer, randomVerb, randomType,randomID, time)
        statements.append(statement)
    save_generated_data(statements, verbs, types)
    end_time=datetime.now()
    delta=end_time-start_time
    with output:
        print("End the data generation")
        print("Generated in "+str(delta))
        clear_output(wait=True)

buttonAccessible=widgets.Button(description='Generate Data')
buttonAccessible.on_click(generate_accessibles)

tabAccessible=VBox([
    HBox([
        checkboxes_verbs_accessibles.multi_select,
        checkboxes_types_accessibles.multi_select,
        vbox_nb_traces_ID_player
    ]),
    buttonAccessible
])

**Alternatives**

In [10]:
verbs_alternatives=["selected", "unlocked"]
checkboxes_verbs_alternatives=GeneratorMultiCheckboxWidget("Aternatives verbs", verbs_alternatives)

types_alternatives=["question", "menu", "dialog", "path", "arena", "alternative"]
checkboxes_types_alternatives=GeneratorMultiCheckboxWidget("Aternatives types", types_alternatives)

template_result_alternative_response={
	"response": None,
	"success": None
}

def generate_alternatives(change):
    start_time=datetime.now()
    with output:
        print("Starting the data generation")
    verbs=checkboxes_verbs_alternatives.get_selected_options()
    types=checkboxes_types_alternatives.get_selected_options()
    ids=generate_IDs()
    times=generate_times_datas()
    player=generate_players()
    statements=[]
    checkTypes={}
    checkAnswer={}
    for i in range(nbtraces.value):
        randomID=random.choice(ids)
        randomPlayer=random.choice(player)
        randomVerb=random.choice(verbs)
        randomType = checkType(checkTypes,randomID,randomPlayer,types)
        time=times[i]
        statement=generator(randomPlayer, randomVerb, randomType,randomID, time)
        if randomType not in checkAnswer.keys():
            checkAnswer[randomType]={}
        if randomID not in checkAnswer[randomType].keys():
            checkAnswer[randomType][randomID]=str(uuid.uuid4())
        if randomType=="alternative":
            result=copy.deepcopy(template_result_alternative_response)
            sucess=random.choice([True, False])
            if sucess:
                result["response"]=checkAnswer[randomType][randomID]
            else:
                result["response"]=str(uuid.uuid4())
            result["success"]=sucess
            statement["result"]=result
        statements.append(statement)
    save_generated_data(statements, verbs, types)
    end_time=datetime.now()
    delta=end_time-start_time
    with output:
        print("End the data generation")
        print("Generated in "+str(delta))
        clear_output(wait=True)

buttonAlternative=widgets.Button(description='Generate Data')
buttonAlternative.on_click(generate_alternatives)

tabAlternatives=VBox([
    HBox([
        checkboxes_verbs_alternatives.multi_select,
        checkboxes_types_alternatives.multi_select,
        vbox_nb_traces_ID_player
    ]),
    buttonAlternative
])

**GameObjects**

In [11]:
verbs_gameObjects=["interacted", "used"]
checkboxes_verbs_gameObjects=GeneratorMultiCheckboxWidget("GameObjects verbs", verbs_gameObjects)

types_gameObjects=["enemy", "npc", "item", "gameobject"]
checkboxes_types_gameObjects=GeneratorMultiCheckboxWidget("GameObjects types", types_gameObjects)

action_types=["talk_to", "examine", "use", "drag_to", "custom"]
checkboxes_action_types=GeneratorMultiCheckboxWidget("ActionTypes", action_types)

template_result_action_type_interact={
    "extensions": {
        "action_type": None,
    }
}

def generate_GameObjects(change):
    start_time=datetime.now()
    with output:
        print("Starting the data generation")
    verbs=checkboxes_verbs_gameObjects.get_selected_options()
    types=checkboxes_types_gameObjects.get_selected_options()
    actionTypes=checkboxes_action_types.get_selected_options()
    ids=generate_IDs()
    times=generate_times_datas()
    player=generate_players()
    statements=[]
    checkTypes={}
    checkActionType={}
    for i in range(nbtraces.value):
        randomID=random.choice(ids)
        randomPlayer=random.choice(player)
        randomVerb=random.choice(verbs)
        randomType= checkType(checkTypes,randomID,randomPlayer,types)
        time=times[i]
        statement=generator(randomPlayer, randomVerb, randomType,randomID, time)
        if randomVerb=="interacted":
            if randomType not in checkActionType.keys():
                checkActionType[randomID]=random.choice([True, False])
            generateActionType=checkActionType[randomID]
            if generateActionType and not len(actionTypes)==0:
                actiontype=random.choice(actionTypes)
                if actiontype=="custom":
                    actiontype=str(uuid.uuid4())
                    actionTypes.append(actiontype)
                result_action_type=copy.deepcopy(template_result_action_type_interact)
                result_action_type["extensions"]["action_type"]=actiontype
                statement["result"]=result_action_type
        statements.append(statement)
    save_generated_data(statements, verbs, types)
    end_time=datetime.now()
    delta=end_time-start_time
    with output:
        print("End the data generation")
        print("Generated in "+str(delta))
        clear_output(wait=True)

buttonGameObjects=widgets.Button(description='Generate Data')
buttonGameObjects.on_click(generate_GameObjects)

tabgameObjects=VBox([
    HBox([
        checkboxes_verbs_gameObjects.multi_select,
        checkboxes_types_gameObjects.multi_select,
        checkboxes_action_types.multi_select
    ]),
    HBox([
        vbox_nb_traces_ID_player,
        buttonGameObjects
    ])
])

**User interface**

In [12]:
types=["Completables", "Accessibles", "Alternatives", "GameObjects"]

tabs=widgets.Tab([tabCompletable, tabAccessible, tabAlternatives, tabgameObjects])

def clearoutput(change):
    with output:
        clear_output(wait=False)

tabs.observe(clearoutput, 'selected_index')

for i in range(len(types)):
    tabs.set_title(i, types[i])
    i+=1

display(tabs)
display(output)

Output()